In [1]:
import math
import ee
import geemap
ee.Authenticate()
ee.Initialize()
geemap.ee_initialize()

#initial surface water mask and additional data
jrcYearly = ee.ImageCollection('JRC/GSW1_4/YearlyHistory')
sword = ee.FeatureCollection('projects/gee-book/assets/A2-4/SWORD')
aoi = ee.Geometry.Polygon([
    [
        [-67.18414102495456, -11.402427204567534],
        [-66.57886300981784, -11.402427204567534],
        [-66.57886300981784, -11.09095257894929],
        [-67.18414102495456, -11.09095257894929],
        [-67.18414102495456, -11.402427204567534]
    ]
])

In [2]:
#given lon & lat in decimal degrees, return ESPG string for the UTM projection
def getUTMProj(lon,lat):
    utmCode = ee.Number(lon).add(180).divide(6).ceil().int()
    output = ee.Algorithms.If(
        condition = ee.Number(lat).gte(0),
        trueCase = ee.String('EPSG:326').cat(utmCode.format('%02d')),
        falseCase = ee.String('EPSG:327').cat(utmCode.format('%02d'))
    )

    return output

#values for aoi
coords = aoi.centroid(30).coordinates()
lon = coords.get(0)
lat = coords.get(1)
crs = getUTMProj(lon, lat)
scale = 30

#reproject data to aoi
def rpj(image):
    return image.reproject(crs = crs.getInfo(), scale = scale)

#length of each pixel (30m*30m)
distanceKernel = ee.Kernel.euclidean(
    radius=30, 
    units = 'meters', 
    magnitude = 0.5
)

def makeChannelmask(year):
    #image closure operation to fill small holes
    watermask = jrcYearly.filter(ee.Filter.eq('year', year)).first().gte(2).unmask(0).focal_max().focal_min().rename('watermask')

    #identify small bars and fill them in
    barPolys = watermask.Not().selfMask().reduceToVectors(
        geometry = aoi, 
        scale = 30, 
        eightConnected = True
    ).filter(ee.Filter.lte('count', 10000))  #get small polys
    filled = watermask.paint(barPolys, 1).rename('filled')

    #cumultive cost mapping to find pixels connected to a centerline
    costmap = filled.Not().cumulativeCost(
        source = watermask.And(ee.Image().toByte().paint(sword, 1)),
        maxDistance = 3000,
        geodeticDistance = False
    ).rename('costmap')

    rivermask = costmap.eq(0).rename('rivermask')
    channelmask = rivermask.And(watermask).rename('channelmask')

    #dilate channelmask to identify banks
    bankMask = channelmask.focal_max(1).neq(channelmask).rename('bankMask')

    #aspect/compass direction of bank faces based on bank distances
    bankDistance = channelmask.Not().cumulativeCost(
        source = channelmask,
        maxDistance = 100,
        geodeticDistance = False
    )
    bankAspect = ee.Terrain.aspect(bankDistance).multiply(math.pi).divide(180).mask(bankMask).rename('bankAspect')
    
    #smoothes bankMask based on distanceKernel; remasks smoothed image to bankMask
    bankLength = bankMask.convolve(distanceKernel).mask(bankMask).rename('bankLength')

    #combining all generated characteristics into one image
    return ee.Image.cat([
        watermask, channelmask, rivermask, bankMask, bankAspect, bankLength
    ]).set('year', year).clip(aoi)

In [16]:
#sample years to calculate change (can change to list: masks = ee.List.sequence(1990,2020,5).map(makeChannelmask))
masks1= makeChannelmask(2015)
masks2 = makeChannelmask(2020)

#pixels that are now the river channel but were previously land
#NOTE: erosion and masks set to pixel units, but can change to area with: ee.Image.pixelArea()
erosion = masks2.select('channelmask').And(masks1.select('watermask').Not()).rename('erosion')

#erosion distance assuming shortest distance between banks
erosionEndpoints = erosion.focal_max(1).And(masks2.select('bankMask'))
erosionDistance= erosion.focal_max(1).selfMask().cumulativeCost(
    source = erosionEndpoints,
    maxDistance = 1000,
    geodeticDistance = True
).rename('erosionDistance')

#erosion direction following slope of distance
erosionDirection = ee.Terrain.aspect(erosionDistance).clip(aoi).rename('erosionDirection').updateMask(erosion)
erosionDistance = erosionDistance.mask(erosion)

#distance to nearest SWORD centerline point
distance = sword.distance(2000).clip(aoi)

#second derivatives of distance, finding the 0s identifies boundaries between centerline points
concavityBounds = distance.convolve(ee.Kernel.laplacian8()).gte(0).rename('bounds')

#reduce pixels according to concavity boundaries; set value to SWORD node ID; focalMode to fill empty pixels that were boundaries
swordImg = ee.Image(0).paint(sword, 'node_id').rename('node_id').clip(aoi)
nodePixels = concavityBounds.addBands(swordImg).reduceConnectedComponents(
    reducer = ee.Reducer.max(),
    labelBand = 'bounds'
).focalMode(
    radius = 3,
    iterations=2
)

In [68]:
#data summary
def groupReduce(dataImg, nodeIds, reducer):
    groupReducer = reducer.forEach(dataImg.bandNames()).group(
        groupField = dataImg.bandNames().length(),
        groupName = 'node_id'
    )

    #apply grouped reducer
    statsList = dataImg.addBands(nodeIds).clip(aoi).reduceRegion(
        reducer = groupReducer,
        maxPixels = 1e8,
        scale = 30
    ).get('groups')

    #convert dictionaries to FeatureCollection
    statsOut = ee.List(statsList).map(lambda d: ee.Feature(None, d))
    return ee.FeatureCollection(statsOut)

#reducer to get total number of pixels each (distances)
dataMask = masks1.addBands(masks2).reduce(ee.Reducer.anyNonZero())

sumBands = ['watermask', 'channelmask', 'bankLength']
sumImg = erosion.addBands(masks1, sumBands).addBands(masks2, sumBands)
sumStats = groupReduce(sumImg, nodePixels, ee.Reducer.sum()) #ee.Reducer.circularMean() requires inputs to be in radians

#reducer to find mean direction
angleBands = ['bankAspect', 'erosionDirection']  # Include erosionDirection
angleImg = masks1.select(['bankAspect']).addBands(masks2.select(['bankAspect'])).addBands(erosionDirection)
angleStats = groupReduce(angleImg, nodePixels, ee.Reducer.circularMean())

#join the 2 feature collections to original centerline data; modify angle data from radians to degrees
def copyProperties(feat):
    nodeFilter = ee.Filter.eq('node_id', feat.get('node_id'))
    sumFeat = sumStats.filter(nodeFilter).first()
    angleFeat = angleStats.filter(nodeFilter).first()
    
    feat = feat.copyProperties(sumFeat).copyProperties(angleFeat)
    if angleFeat:
        bankAspectRad = ee.Number(angleFeat.get('bankAspect'))
        bankAspectRad1 = ee.Number(angleFeat.get('bankAspect_1'))
        erosionDirectionRad = ee.Number(angleFeat.get('erosionDirection'))
 
        #if not NaN convert from radians to degrees
        bankAspectDeg = ee.Algorithms.If(
            ee.Algorithms.IsEqual(bankAspectRad, None),
            None,
            bankAspectRad.multiply(180).divide(math.pi)
        )
        bankAspectDeg1 = ee.Algorithms.If(
            ee.Algorithms.IsEqual(bankAspectRad1, None),
            None,
            bankAspectRad1.multiply(180).divide(math.pi)
        )
        erosionDirectionDeg = ee.Algorithms.If(
            ee.Algorithms.IsEqual(erosionDirectionRad, None),
            None,
            erosionDirectionRad.multiply(180).divide(math.pi)
        )
        
        #make angles positive
        bankAspectDeg = ee.Algorithms.If(
            ee.Algorithms.IsEqual(bankAspectDeg, None),
            None,
            ee.Algorithms.If(
                ee.Number(bankAspectDeg).lt(0),
                ee.Number(bankAspectDeg).add(360),
                bankAspectDeg
            )
        )
        bankAspectDeg1 = ee.Algorithms.If(
            ee.Algorithms.IsEqual(bankAspectDeg1, None),
            None,
            ee.Algorithms.If(
                ee.Number(bankAspectDeg1).lt(0),
                ee.Number(bankAspectDeg1).add(360),
                bankAspectDeg1
            )
        )
        erosionDirectionDeg = ee.Algorithms.If(
            ee.Algorithms.IsEqual(erosionDirectionDeg, None),
            None,
            ee.Algorithms.If(
                ee.Number(erosionDirectionDeg).lt(0),
                ee.Number(erosionDirectionDeg).add(360),
                erosionDirectionDeg
            )
        )
        
        feat = feat.set('bankAspect', bankAspectDeg)
        feat = feat.set('bankAspect_1', bankAspectDeg1)
        feat = feat.set('erosionDirection', erosionDirectionDeg)
    return feat

#apply new properties to sword nodes
vectorData = sword.filterBounds(aoi).map(copyProperties)

#print final FeatureCollection
#print(vectorData.limit(3).getInfo())

#export data as CSV to Drive
export = ee.batch.Export.table.toDrive(collection=vectorData, description='RiverErosion_2015_2020', fileFormat='CSV')
export.start()


In [74]:
#initialize map for rendering
Map = geemap.Map()
Map.centerObject(aoi, 13)

In [75]:
#layers
year1mask = rpj(masks1.select('channelmask').selfMask())
Map.addLayer(year1mask, {'palette': ['blue']}, '2015')
year2mask = rpj(masks2.select('channelmask').selfMask())
Map.addLayer(year2mask, {'palette': ['red']}, '2020', True, 0.5)
Map.addLayer(rpj(erosion).selfMask(), {'palette': ['red']}, 'erosion', True)
Map.addLayer(rpj(nodePixels).randomVisualizer(), {}, 'node assignments', True)
#Map.addLayer(rpj(distance), {'min': 0, 'max': 1000}, 'distance', True)
#Map.addLayer(rpj(erosionDistance), {'min': 0, 'max':300}, 'erosion distance', True)
#Map.addLayer(rpj(erosionDirection), {'min': 0, 'max': math.pi}, 'erosion direction', True)
Map.addLayer(vectorData, {}, 'final data nodes')

Map.addLayerControl()
Map

Map(center=[-11.246814724383926, -66.881502017386], controls=(WidgetControl(options=['position', 'transparent_…